# 🧱 Structuring Data: Practicing Tidy Data

*... and taking the next step.*

In [1]:
import pandas as pd

In [2]:
billboard_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vScPt_dOJIulsY96YvXYVVR4PZWSpJfMJKjjJowaz_P_bwRfkAHxVViNG8_mm7Dpc_44bvLO0cwkfLD/pub?gid=713819209&single=true&output=csv"
billboard = pd.read_csv(billboard_url)
billboard

,artist.inverted,track,date.entered,x1st.week,x2nd.week,x3rd.week
0,"Houston, Whitney",My Love Is Your Love,1999-09-04,81,68,44
1,Jay-Z,I Just Wanna Love U (Give It 2 Me),2000-10-28,58,45,35
2,Jay-Z,Big Pimpin',2000-04-22,69,52,39
3,"Houston, Whitney",I Learned From The Best,2000-02-19,83,83,83
4,"Houston, Whitney",Could I Have This Kiss Forever,2000-06-17,74,68,68
5,Jay-Z,Anything,2000-02-26,72,58,55
6,Jay-Z,Do It Again (Put Ya Hands Up),2000-01-15,95,68,65
7,"Houston, Whitney","Same Script, Different Cast",2000-06-17,71,71,71
8,Jay-Z,Hey Papi,2000-08-12,98,100,98


## Questions: 

- What is the unit of observation?
- What are the variables?
- How would we tidy this?

In [3]:
billboard_semitidy = (
    billboard
    .melt(
        id_vars = [ "artist.inverted", "track", "date.entered" ],
        value_name = "rank"
    )
    .assign(
        week_num = lambda df: df["variable"].str.extract(r"(\d+)").astype(int)
    )
    .drop(columns = ["variable"])
)

billboard_semitidy

,artist.inverted,track,date.entered,rank,week_num
0,"Houston, Whitney",My Love Is Your Love,1999-09-04,81,1
1,Jay-Z,I Just Wanna Love U (Give It 2 Me),2000-10-28,58,1
2,Jay-Z,Big Pimpin',2000-04-22,69,1
3,"Houston, Whitney",I Learned From The Best,2000-02-19,83,1
4,"Houston, Whitney",Could I Have This Kiss Forever,2000-06-17,74,1
5,Jay-Z,Anything,2000-02-26,72,1
6,Jay-Z,Do It Again (Put Ya Hands Up),2000-01-15,95,1
7,"Houston, Whitney","Same Script, Different Cast",2000-06-17,71,1
8,Jay-Z,Hey Papi,2000-08-12,98,1
9,"Houston, Whitney",My Love Is Your Love,1999-09-04,68,2


In [4]:
(
    billboard_semitidy
    .groupby([ "artist.inverted" ])
    ["rank"]
    .min()
)

artist.inverted
Houston, Whitney    44
Jay-Z               35
Name: rank, dtype: int64

## Separating the observational units

(What *are* the two observational units?)

- __Track__ (track name, artist, debut date)
- __Track appearance__ (track, week, rank)

To separate those units, we'll need to create a unique ID for each track, so that we can join the information back together. One way is with `.reset_index(...)`:

In [5]:
(
    billboard
    .reset_index(names="song_id")
)

,song_id,artist.inverted,track,date.entered,x1st.week,x2nd.week,x3rd.week
0,0,"Houston, Whitney",My Love Is Your Love,1999-09-04,81,68,44
1,1,Jay-Z,I Just Wanna Love U (Give It 2 Me),2000-10-28,58,45,35
2,2,Jay-Z,Big Pimpin',2000-04-22,69,52,39
3,3,"Houston, Whitney",I Learned From The Best,2000-02-19,83,83,83
4,4,"Houston, Whitney",Could I Have This Kiss Forever,2000-06-17,74,68,68
5,5,Jay-Z,Anything,2000-02-26,72,58,55
6,6,Jay-Z,Do It Again (Put Ya Hands Up),2000-01-15,95,68,65
7,7,"Houston, Whitney","Same Script, Different Cast",2000-06-17,71,71,71
8,8,Jay-Z,Hey Papi,2000-08-12,98,100,98


Another way would be to create an unique string:

In [6]:
(
    billboard
    .assign(
        song_id = lambda df: (
            (
                df["artist.inverted"].str.slice(0, 3) + ":" + 
                df["track"].str.slice(0, 3)
            )
            .str.lower()
            .str.replace(" ", "#")
        )
    )
)

,artist.inverted,track,date.entered,x1st.week,x2nd.week,x3rd.week,song_id
0,"Houston, Whitney",My Love Is Your Love,1999-09-04,81,68,44,hou:my#
1,Jay-Z,I Just Wanna Love U (Give It 2 Me),2000-10-28,58,45,35,jay:i#j
2,Jay-Z,Big Pimpin',2000-04-22,69,52,39,jay:big
3,"Houston, Whitney",I Learned From The Best,2000-02-19,83,83,83,hou:i#l
4,"Houston, Whitney",Could I Have This Kiss Forever,2000-06-17,74,68,68,hou:cou
5,Jay-Z,Anything,2000-02-26,72,58,55,jay:any
6,Jay-Z,Do It Again (Put Ya Hands Up),2000-01-15,95,68,65,jay:do#
7,"Houston, Whitney","Same Script, Different Cast",2000-06-17,71,71,71,hou:sam
8,Jay-Z,Hey Papi,2000-08-12,98,100,98,jay:hey


## First, let's create a table of *just* the track-level variables

In [7]:
billboard_tracks = (
    billboard
    .reset_index(names="song_id")
    [[
        "song_id",
        "artist.inverted",
        "track",
        "date.entered"
    ]]
)

billboard_tracks

,song_id,artist.inverted,track,date.entered
0,0,"Houston, Whitney",My Love Is Your Love,1999-09-04
1,1,Jay-Z,I Just Wanna Love U (Give It 2 Me),2000-10-28
2,2,Jay-Z,Big Pimpin',2000-04-22
3,3,"Houston, Whitney",I Learned From The Best,2000-02-19
4,4,"Houston, Whitney",Could I Have This Kiss Forever,2000-06-17
5,5,Jay-Z,Anything,2000-02-26
6,6,Jay-Z,Do It Again (Put Ya Hands Up),2000-01-15
7,7,"Houston, Whitney","Same Script, Different Cast",2000-06-17
8,8,Jay-Z,Hey Papi,2000-08-12


## Now, just the appearance-level variables

In [8]:
billboard_ranks = (
    billboard
    .reset_index(names="song_id")
    .drop(columns = [ "artist.inverted", "track", "date.entered" ])
    .melt(
        id_vars = [ "song_id" ],
        value_name = "rank"
    )
    .assign(
        week_num = lambda df: df["variable"].str.extract(r"(\d+)").astype(int)
    )
    .drop(columns = ["variable"])    
)

billboard_ranks.head()

,song_id,rank,week_num
0,0,81,1
1,1,58,1
2,2,69,1
3,3,83,1
4,4,74,1


## We can still join them back together, using our `song_id`

In [9]:
(
    billboard_tracks
    .merge(
        billboard_ranks,
        on="song_id"
    )
    .head()
)

,song_id,artist.inverted,track,date.entered,rank,week_num
0,0,"Houston, Whitney",My Love Is Your Love,1999-09-04,81,1
1,0,"Houston, Whitney",My Love Is Your Love,1999-09-04,68,2
2,0,"Houston, Whitney",My Love Is Your Love,1999-09-04,44,3
3,1,Jay-Z,I Just Wanna Love U (Give It 2 Me),2000-10-28,58,1
4,1,Jay-Z,I Just Wanna Love U (Give It 2 Me),2000-10-28,45,2


## But now we can do things that we couldn't do with the semi-tidy data

In [10]:
(
    billboard_tracks
    ["artist.inverted"]
    .value_counts()
)

artist.inverted
Jay-Z               5
Houston, Whitney    4
Name: count, dtype: int64

## ... and we can construct our analyses more flexibly

In [11]:
(
    billboard_ranks
    .groupby("song_id")
    ["rank"]
    .min()
    .sort_values()
    .to_frame("rank_best")
    .reset_index()
    .merge(
        billboard_tracks
        [[
            "song_id",
            "track",
            "artist.inverted"
        ]]
    )
)

,song_id,rank_best,track,artist.inverted
0,1,35,I Just Wanna Love U (Give It 2 Me),Jay-Z
1,2,39,Big Pimpin',Jay-Z
2,0,44,My Love Is Your Love,"Houston, Whitney"
3,5,55,Anything,Jay-Z
4,6,65,Do It Again (Put Ya Hands Up),Jay-Z
5,4,68,Could I Have This Kiss Forever,"Houston, Whitney"
6,7,71,"Same Script, Different Cast","Houston, Whitney"
7,3,83,I Learned From The Best,"Houston, Whitney"
8,8,98,Hey Papi,Jay-Z


# Now with the full dataset ...

In [12]:
billboard_full_url = "https://github.com/hadley/tidy-data/raw/master/data/billboard.csv"
billboard_full = pd.read_csv(billboard_full_url, encoding="latin-1")
billboard_full.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
billboard_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 83 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             317 non-null    int64  
 1   artist.inverted  317 non-null    object 
 2   track            317 non-null    object 
 3   time             317 non-null    object 
 4   genre            317 non-null    object 
 5   date.entered     317 non-null    object 
 6   date.peaked      317 non-null    object 
 7   x1st.week        317 non-null    int64  
 8   x2nd.week        312 non-null    float64
 9   x3rd.week        307 non-null    float64
 10  x4th.week        300 non-null    float64
 11  x5th.week        292 non-null    float64
 12  x6th.week        280 non-null    float64
 13  x7th.week        269 non-null    float64
 14  x8th.week        260 non-null    float64
 15  x9th.week        253 non-null    float64
 16  x10th.week       244 non-null    float64
 17  x11th.week      

In [14]:
billboard_id_vars = [
    "year",
    "artist.inverted",
    "track",
    "time",
    "genre",
    "date.entered",
    "date.peaked"
]

In [15]:
billboard_full_semitidy = (
    billboard_full
    .melt(
        id_vars = billboard_id_vars,
        value_name = "rank"
    )
    # Two new lines here: Let's see why
    .dropna(subset=["rank"])
    .astype({ "rank": int })
    .assign(
        week_num = lambda df: df["variable"].str.extract(r"(\d+)").astype(int)
    )
    .drop(columns = ["variable"])
)

billboard_full_semitidy

,year,artist.inverted,track,time,genre,date.entered,date.peaked,rank,week_num
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,1
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,1
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,1
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,1
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,1
...,...,...,...,...,...,...,...,...,...
19663,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,45,63
19700,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,50,63
19980,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,50,64
20017,2000,Creed,Higher,5:16,Rock,1999-09-11,2000-07-22,50,64


## Exersise: Create the fully-tidy version of this full dataset

... separating track-level and appearance-level data into their own tables.

---

---

---